In [3]:
import psycopg2
from django.db import transaction
transaction.rollback()

In [5]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BinanceDataAPI.settings")
import django
django.setup()
#from binance.client import Client
from datetime import datetime
import requests

# TESTANDO COM WRAP PYTHON E REQUEST

In [2]:
#Binance Client

#Para obtenção destes dados, não é necessário autenticar usuário
client = Client('', '')

In [58]:
#candles = client.get_klines(symbol='BNBBTC', interval=Client.KLINE_INTERVAL_1MINUTE)

request_machine_instant = datetime.now()

#request_binance_instant = datetime.fromtimestamp(int(float(client.get_server_time()['serverTime'])/1000))

symbol = 'BNBBTC'
granulacao_malha = '1m'

r = requests.get('https://api.binance.com/api/v1/klines?symbol='+symbol+'&interval='+granulacao_malha)
#Retorna do candle aberto mais recente até 499 candles anteriores, por request
'''
    1499040000000,      // Open time (miliseconds)
    "0.01634790",       // Open
    "0.80000000",       // High
    "0.01575800",       // Low
    "0.01577100",       // Close
    "148976.11427815",  // Volume
    1499644799999,      // Close time
    "2434.19055334",    // Quote asset volume
    308,                // Number of trades
    "1756.87402397",    // Taker buy base asset volume
    "28.46694368",      // Taker buy quote asset volume
    "17928899.62484339" // Ignore.
  ]

'''

'\n    1499040000000,      // Open time (miliseconds)\n    "0.01634790",       // Open\n    "0.80000000",       // High\n    "0.01575800",       // Low\n    "0.01577100",       // Close\n    "148976.11427815",  // Volume\n    1499644799999,      // Close time\n    "2434.19055334",    // Quote asset volume\n    308,                // Number of trades\n    "1756.87402397",    // Taker buy base asset volume\n    "28.46694368",      // Taker buy quote asset volume\n    "17928899.62484339" // Ignore.\n  ]\n\n'

In [62]:
print('Candle mais antigo retornado:')
      
print('> Wrap....:',candles[0])
print ('')
print('> Request.:',r.json()[0])
print ('')
print ('')

print('Candle mais recente retornado:')
      
print('> Wrap....:',candles[499])
print ('')
print('> Request.:',r.json()[499])

Candle mais antigo retornado:
> Wrap....: [1557223560000, '0.00372330', '0.00372710', '0.00372330', '0.00372340', '390.43000000', 1557223619999, '1.45417279', 38, '103.62000000', '0.38608025', '0']

> Request.: [1557223560000, '0.00372330', '0.00372710', '0.00372330', '0.00372340', '390.43000000', 1557223619999, '1.45417279', 38, '103.62000000', '0.38608025', '0']


Candle mais recente retornado:
> Wrap....: [1557253500000, '0.00365330', '0.00365980', '0.00365300', '0.00365430', '247.76000000', 1557253559999, '0.90587357', 28, '202.68000000', '0.74101787', '0']

> Request.: [1557253500000, '0.00365330', '0.00365980', '0.00365300', '0.00365430', '247.76000000', 1557253559999, '0.90587357', 28, '202.68000000', '0.74101787', '0']


In [70]:
#São retornados 500 candles por request
print(len(candles))
print(len(r.json()))

500
500


In [11]:
def MinToHour(minutos):
    
    H = str(int(minutos/60))
    m = str(minutos%60)
    
    horas = H+':'+m
    
    return horas

In [36]:
#Teste

candle_mais_antigo_retornado = datetime.fromtimestamp(int((candles[0][0])/1000))

candle_mais_novo_retornado = datetime.fromtimestamp(int((candles[499][0])/1000))

print("REQUEST INSTANT:")
print ('')
print("> local: ",request_machine_instant)
print ('')
print("> Binance: ",request_binance_instant)
print ('')
print ('candle_mais_antigo_retornado = ',candle_mais_antigo_retornado)
print ('')
print('candle_mais_novo_retornado = ',candle_mais_novo_retornado)
print ('')

delta = MinToHour(500)

print('Diferença de tempo entre os candles = ' + delta)

#Será necessário descartar a última amostra, já que o candle deste instante ainda não foi fechado
#O horário da binance está em UTC

REQUEST INSTANT:

> local:  2019-05-07 17:20:03.758597

> Binance:  2019-05-07 17:20:26

candle_mais_antigo_retornado =  2019-05-07 09:01:00

candle_mais_novo_retornado =  2019-05-07 17:20:00

Diferença de tempo entre os candles = 8:20


In [82]:
# Agora é preciso repetir o request definindo instantes limites da aquisição

symbol = 'BNBBTC'
granulacao_malha = '1m'
start_time = '1557223560000'
end_time = '1557253500000'

# 1 - Com ambos os limites definidos 
r_1 = requests.get('https://api.binance.com/api/v1/klines?symbol='+symbol+'&interval='+granulacao_malha+'&startTime='+start_time+'&endTime='+end_time)

# 2 - Apenas com limite inferior definido 
r_2 = requests.get('https://api.binance.com/api/v1/klines?symbol='+symbol+'&interval='+granulacao_malha+'&startTime='+start_time)

# 3 - Apenas com limite superior definido 
r_3 = requests.get('https://api.binance.com/api/v1/klines?symbol='+symbol+'&interval='+granulacao_malha+'&endTime='+end_time)


In [87]:
# Testando consistência do dado para r_1

print('Candle mais antigo retornado:')
      
print('> Wrap....:',candles[0])
print ('')
print('> Request.:',r_1.json()[0])
print ('')
print ('')

print('Candle mais recente retornado:')
      
print('> Wrap....:',candles[499])
print ('')
print('> Request.:',r_1.json()[499])

#OBS: Os dados são os mesmos, embora neste cenário o último candle já esteja, obviamente, 
#fechado, o que faz divergir os últimos valores ("Quote asset volume", "Number of trades", "Taker buy 
#base asset volume", "Taker buy quote asset volume" e "Ignore".)

Candle mais antigo retornado:
> Wrap....: [1557223560000, '0.00372330', '0.00372710', '0.00372330', '0.00372340', '390.43000000', 1557223619999, '1.45417279', 38, '103.62000000', '0.38608025', '0']

> Request.: [1557223560000, '0.00372330', '0.00372710', '0.00372330', '0.00372340', '390.43000000', 1557223619999, '1.45417279', 38, '103.62000000', '0.38608025', '0']


Candle mais recente retornado:
> Wrap....: [1557253500000, '0.00365330', '0.00365980', '0.00365300', '0.00365430', '247.76000000', 1557253559999, '0.90587357', 28, '202.68000000', '0.74101787', '0']

> Request.: [1557253500000, '0.00365330', '0.00365980', '0.00365300', '0.00365790', '248.27000000', 1557253559999, '0.90773909', 29, '203.19000000', '0.74288339', '0']


In [88]:
# Testando consistência do dado para r_2

print('Candle mais antigo retornado:')
      
print('> Wrap....:',candles[0])
print ('')
print('> Request.:',r_2.json()[0])
print ('')
print ('')

print('Candle mais recente retornado:')
      
print('> Wrap....:',candles[499])
print ('')
print('> Request.:',r_2.json()[499])

#OBS: Os dados são os mesmos, embora neste cenário o último candle já esteja, obviamente, 
#fechado, o que faz divergir os últimos valores ("Quote asset volume", "Number of trades", "Taker buy 
#base asset volume", "Taker buy quote asset volume" e "Ignore".)

Candle mais antigo retornado:
> Wrap....: [1557223560000, '0.00372330', '0.00372710', '0.00372330', '0.00372340', '390.43000000', 1557223619999, '1.45417279', 38, '103.62000000', '0.38608025', '0']

> Request.: [1557223560000, '0.00372330', '0.00372710', '0.00372330', '0.00372340', '390.43000000', 1557223619999, '1.45417279', 38, '103.62000000', '0.38608025', '0']


Candle mais recente retornado:
> Wrap....: [1557253500000, '0.00365330', '0.00365980', '0.00365300', '0.00365430', '247.76000000', 1557253559999, '0.90587357', 28, '202.68000000', '0.74101787', '0']

> Request.: [1557253500000, '0.00365330', '0.00365980', '0.00365300', '0.00365790', '248.27000000', 1557253559999, '0.90773909', 29, '203.19000000', '0.74288339', '0']


In [86]:
# Testando consistência do dado para r_3

print('Candle mais antigo retornado:')
      
print('> Wrap....:',candles[0])
print ('')
print('> Request.:',r_3.json()[0])
print ('')
print ('')

print('Candle mais recente retornado:')
      
print('> Wrap....:',candles[499])
print ('')
print('> Request.:',r_3.json()[498])

# Neste cenário, as entradas divergiram a partir do volume

Candle mais antigo retornado:
> Wrap....: [1557223560000, '0.00372330', '0.00372710', '0.00372330', '0.00372340', '390.43000000', 1557223619999, '1.45417279', 38, '103.62000000', '0.38608025', '0']

> Request.: [1557223560000, '0.00372330', '0.00372710', '0.00372330', '0.00372340', '390.43000000', 1557223619999, '1.45417279', 38, '103.62000000', '0.38608025', '0']


Candle mais recente retornado:
> Wrap....: [1557253500000, '0.00365330', '0.00365980', '0.00365300', '0.00365430', '247.76000000', 1557253559999, '0.90587357', 28, '202.68000000', '0.74101787', '0']

> Request.: [1557253440000, '0.00365970', '0.00365980', '0.00365190', '0.00365510', '1244.03000000', 1557253499999, '4.54674412', 25, '18.54000000', '0.06776524', '0']


In [7]:
datetime.fromtimestamp(1557253500)

datetime.datetime(2019, 5, 7, 15, 25)

In [10]:
1557253500000-315360000000

1241893500000

In [11]:
datetime.fromtimestamp(1241893500)

datetime.datetime(2009, 5, 9, 15, 25)

# BinanceDataAPI

##### Um projeto django para aquisição, tratamento e servimento de dados dos mercados de criptomoedas da corretora Binance.  

> O objetivo deste projeto/serviço é sempre dispor de banco de dados próprio, qualquer que seja o ativo operado (desde que esteja sendo - ou tenha sido - operado ), com dados aramzenados na escala mais refinada disponibizada pela corretora (neste caso, 1m).

> Deve tratar do controle de requests junto ao provedor dos dados, a fim de não esceder limites estabelecidos.

> Deve servir os dados em outras escalas (tendo estes fidalidade absoluta com aqueles de respectiva escala, originais, disponibilzados pela corretora), bem como seus indicadores associados, como uma API.

> Espera-se que isto traga alguma otimização ao paradgma, já que que qualquer modelagem será possível, a partir de uma refinada massa de dados offline, instanciada em banco de dados próprio, que é uma boa prática de saída. Assim, modelagens em outras escalas temporais - candle de 15m, por exemplo - são possíveis sem novos requests à corretora e sem novo arquivamento de uma nova massa de dados, que poderia ser obtida - por lógica - a partir da primeira.

## Estrutura

###  1 - getdata  

#### Aquisição e salvamento dos dados históricos de mercado de criptomoedas, junto à corretora binance.   

> É um processo que deve ocorrer em background

#### Lógica de negócio

**A - Início**
    
    0 - Cosultar o master em busca de quais ativos trabalhar
    
    * Para cada ativo "enable" (ou seja, um subprocesso para cada ativo):
    
    1 - Consultar o banco de dados em busca da tabela referente ao 'symbol' operado;
        
        1.1 - Existindo:
            i) "Completa para frente" até o presente, a partir da amostra mais recente encontrada;
            ii) "Completa para trás", a partir da amostra mais antiga encontrada;
            iii) Atualiza status;
            iv) Cicla.
        
        1.2 Não existindo:
            i) Completa "para trás" a partir do presente;
            ii) Atualiza status;
            iii) Cicla.
 
**B - Ciclos**

    2 - Cosultar o master em busca do status ('enable' ou 'disable') do ativo trabalhado neste subprocesso
    2 - Consultar o banco de dados em busca da tabela referente ao 'symbol' operado;
    3 - Completa pra frente a partir da amostra mais recente encontrada;
    4 - Atualiza status
    5 - Descansa;
    6 - Repete enquanto [ ? ].

**Observações**

    - Deve tratar de maneira lógica o balanço de peso das requests
    - Deve tratar as exceções.
    - O limite de requests por minuto, por ativo, é o inteiro menor mais próximo do resultado da divisão: 1200/N, em que N é o número de ativos "enable".
    - Cada request retorna 500 amostras.
    
### 2 - API

#### Deve responder à solicitações

## Plano de Ação

* [x] Criar o app getdata
* [ ] Criar o app master
* [ ] Criar e alimentar com "valores-modelos" os seguintes model/tabelas:
  * [ ] UMA tabela para cada par, sendo todas as tabelas réplicas de um mesmo modelo (app getdata)
  * [ ] Uma tabela para o master (app getdata)
  * [ ] Uma tabela para
* [ ] Criar a primeira tabela no banco, a partir de um candle, dando entrada pela interface web
* [ ] Escrever e testar as funções de completar para trás e para frente
* [ ] Finalizar este módulo


In [3]:
print(datetime.now())

2019-05-09 14:36:14.969443


# EXEMPLO DE CONVERSÃO DE DADOS PARA DATAFRAME

In [71]:
def data_historico(par,candle,time_aq):
    """
    Função que recupera os dados históricos de um par junto à Binance.
    """
    
    hist1 = client.get_historical_klines(par, candle, time_aq)
    
    open_time = []; Open = []; High = []; Low = []; Close = []; Volume = []
    Close_time = [] ;Quote_asset_volume = []; Number_of_trades = []
    #Taker_buy_base_asset_volume = []; Taker_buy_quote_asset_volume = [] ; Can_be_ignored = []
    for i in range(0,len(hist1)):
        open_time.append(float(hist1[i][0])); Open.append(float(hist1[i][1])); 
        High.append(float(hist1[i][2])); Low.append(float(hist1[i][3]))
        Close.append(float(hist1[i][4])); Volume.append(float(hist1[i][5]))
        Close_time.append(float(hist1[i][6])); #Quote_asset_volume.append(float(hist1[i][7]))
        Number_of_trades.append(float(hist1[i][8]))#; Taker_buy_base_asset_volume.append(float(hist1[i][9]))
        #Taker_buy_quote_asset_volume.append(float(hist1[i][10])); Can_be_ignored.append(float(hist1[i][11]))
    
    ts_open_time = [datetime.fromtimestamp(x/1000) for x in open_time]
    ts_close_time = [datetime.fromtimestamp(x/1000) for x in Close_time]
    
    d = {'Open': Open, 'High': High, 'Low': Low, 'Close': Close, 
         'Close_time': ts_close_time, 'Volume': Volume, 'N_of_trades': Number_of_trades}
    df_hist = pd.DataFrame(data=d, index=ts_open_time)
    
    #'Open_time': ts_open_time, retirei esta coluna do dataframe
    
    return df_hist #open_time,Open,High,Low,Close,Close_time,Volume,Number_of_trades

In [2]:
from django.shortcuts import render, redirect, get_object_or_404
from django.contrib.auth.decorators import login_required
from getdata.models import Pair
from getdata.forms import PairForm

In [48]:
Pair.objects.values_list()

<QuerySet [('ETHUSDT', 'ON'), ('BTCUSDT', 'ON')]>

In [50]:
Pair.objects.get(pair_name='BTCUSCT')['state']

DoesNotExist: Pair matching query does not exist.

In [58]:
Pair.objects.get(pair_name='BTCUSDT')

<Pair: BTCUSDT>